In [2]:
import random
import numpy as np
import math
from sklearn import preprocessing


In [3]:
train_file = '../datasets/optdigits/optdigits_train.csv'
test_file = '../datasets/optdigits/optdigits_test.csv.txt'



In [4]:
keep_labels = [7,1,5]
le = preprocessing.LabelEncoder()
le.fit(keep_labels)
num_hidden = 10

In [13]:
train_lines = open(train_file, 'r').readlines()
#print train_lines

test_lines = open(test_file, 'r').readlines()

print "==> Total no. of samples: %d" %(np.array(train_lines).shape[0] + np.array(test_lines).shape[0])

train_vectors = []
train_labels = []
test_vectors = []
test_labels = []

num_input_nodes = len(train_lines[0].strip().split(',')) - 1

print "==> Number of input nodes: %d" %num_input_nodes
count = 0
for line in train_lines:
    vector = [int(x) for x in line.strip().split(',')]

    train_vectors.append(np.array(vector))
    train_labels.append(np.array(vector))

train_vectors = np.vstack(train_vectors)[:,0:-1]
train_labels =  np.vstack(train_labels)[:,-1]
if keep_labels is not None:
    if len(keep_labels)==2:
        num_output_unit = 1
    else:
        num_output_unit = len(keep_labels)
else:
    num_output_unit =  np.unique(train_labels).shape[0]

weights1 = np.random.randn(num_input_nodes, num_hidden)/float(np.sqrt(num_input_nodes))
b1 = np.zeros((num_hidden))
weights2 = np.random.randn(num_hidden, num_output_unit)/float(np.sqrt(num_hidden))
b2 = np.zeros((num_output_unit))

print "==> Weights Initialized"
for line in test_lines:
    vector = [int(x) for x in line.strip().split(',')]

    test_vectors.append(np.array(vector))
    test_labels.append(np.array(vector))


test_vectors = np.vstack(test_vectors)[:,0:-1]
test_labels =  np.vstack(test_labels)[:,-1]


# Make labels to categorical labels and filter out the labels
if keep_labels is not None:
    where_keep_labels = []	
    for l in keep_labels:
        where_keep_labels.extend(list(np.where(train_labels==l)[0]) )

    n_data = np.sum([wkl.size for wkl in where_keep_labels])
    n_keep_labels = len(keep_labels)

    original_train_vectors = train_vectors.copy()
    train_vectors = np.zeros((n_data,num_input_nodes))

else:
    n_data = train_vectors.shape[0]
    n_keep_labels = np.max(train_labels)+1

train_labels_cat = np.zeros((n_data, n_keep_labels))
filtered_labels = np.zeros((n_data))

count = 0
for i in range(train_labels.shape[0]):
    if keep_labels is not None :
        if train_labels[i] in keep_labels:
            train_vectors[count,:] = original_train_vectors[i,:]

            #self.train_labels_cat[count,self.train_labels[i]- min(keep_labels)] = 1
            filtered_labels[count] = train_labels[i]
            count +=1
    else:
        train_labels_cat[i,train_labels[i]] = 1

train_filtered_labels_encoded = le.transform(filtered_labels)

for i in range(train_filtered_labels_encoded.shape[0]):
    train_labels_cat[i, train_filtered_labels_encoded[i]] = 1

def sigmoid(x):
	return 1/(1 + np.exp(-x))
def predict(train_vectors):
        # Forward Pass
        predictions = []
        for x in range(train_vectors.shape[0]):
            a_1 = sigmoid(np.matmul((weights1).T, train_vectors[x,:])[:,np.newaxis]+self.b1[:,np.newaxis])	

            a_2 = sigmoid(np.matmul(weights2.T, a_1)+b2[:, np.newaxis])
            predictions.append(np.argmax(a_2, axis = 0))


        predictions =  np.vstack(predictions)[:,0] 
        #

        accuracy = np.sum(predictions == self.train_filtered_labels_encoded)/float(np.shape(self.train_filtered_labels_encoded)[0])
        #pdb.set_trace()
        print 
        

def run( max_epochs=100, lr_rate=0.01):

    for i in range(max_epochs):
        train_Samples_num = X_train.shape[0]
        shuffled_indices = random.sample(range(train_Samples_num), train_Samples_num)

        for x in shuffled_indices:

            # Forward Pass
            # a_1 = sigmoid(X_train[x].dot(weights1)+b1)		

            # print weights1.shape, X_train[x].shape, np.matmul((weights1).T, X_train[x]).shape, b1.shape
            a_1 = sigmoid(np.matmul((weights1).T, X_train[x])[:,np.newaxis]+b1[:,np.newaxis])	
            #print a_1.shape, weights2.shape, b2[np.newaxis,:].shape

            # print a_1.shape, weights2.shape

            a_2 = sigmoid(np.matmul(weights2.T, a_1)+b2[:, np.newaxis])
            #print a_1.shape, a_2.shape




            # Backpropagation
            #print (train_labels_cat[x][:, np.newaxis] - a_2).shape, (a_2*(1-a_2)).shape
            # print (train_labels_cat[x][:,np.newaxis].shape), (a_2).shape,(a_2*(1-a_2)).shape			# 	#delta_2 = np.linalg.norm(train_labels_cat[x] - a_2)*((a_2*((1-a_2))))
            # pdb.set_trace()
            delta_2 = (a_2 - train_labels_cat[x][:, np.newaxis] )*(a_2*(1-a_2))
            #print delta_2.shape


            dE_dw_2 = np.matmul(a_1, delta_2.T)

            dE_db_2 = (delta_2)
            # print delta_2



        # # 	delta_1 = (a_1*(1-a_1))*(np.matmul(delta_2,weights2.T))
        # 	#print delta_2[:,np.newaxis].shape, weights2.shape
            delta_1 = (a_1*(1-a_1))*np.matmul(weights2, delta_2)

            dE_dw_1 = np.matmul(X_train[x][:,np.newaxis], delta_1.T)				
        #  	#print delta_1.shape, X_train[x][:,np.newaxis].shape, dE_dw_1.shape
            dE_db_1 = (delta_1)

            print delta_1

            # print dE_db_1.shape, b1.shape

        # 	# Update rules

        # 	#print weights1.shape, dE_dw_1.shape

        # 	print weights1.shape, weights2.shape




            weights1 = weights1 - lr_rate*dE_dw_1
            b1 = b1 - lr_rate*dE_db_1[:,0]	


            weights2 = weights2 - lr_rate*dE_dw_2
            b2 = b2 - lr_rate*dE_db_2[:,0]

        # print b1.shape, b2.shape
        #print b1

        accuracy = predict()        


==> Total no. of samples: 5620
==> Number of input nodes: 64
==> Weights Initialized


In [15]:
train_filtered_labels_encoded

array([2, 1, 1, ..., 0, 0, 2])

In [ ]:
def sigmoid(x):
	return 1/(1 + np.exp(-x))

class MLP(object):
    def __init__(self, training_string, testing_string):

        train_lines = open(training_string, 'r').readlines()
        #print train_lines

        test_lines = open(testing_string, 'r').readlines()

        print "==> Total no. of samples: %d" %(np.array(train_lines).shape[0] + np.array(test_lines).shape[0])

        self.train_vectors = []
        self.train_labels = []
        self.test_vectors = []
        self.test_labels = []

        self.num_input_nodes = len(train_lines[0].strip().split(',')) - 1

        print "==> Number of input nodes: %d" %self.num_input_nodes
        count = 0
        for line in train_lines:
            vector = [int(x) for x in line.strip().split(',')]

            self.train_vectors.append(np.array(vector))
            self.train_labels.append(np.array(vector))

        self.train_vectors = np.vstack(self.train_vectors)[:,0:-1]
        self.train_labels =  np.vstack(self.train_labels)[:,-1]
        if keep_labels is not None:
            if len(keep_labels)==2:
                num_output_unit = 1
            else:
                num_output_unit = len(keep_labels)
        else:
            num_output_unit =  np.unique(self.train_labels).shape[0]

        self.weights1 = np.random.randn(self.num_input_nodes, num_hidden)/float(np.sqrt(self.num_input_nodes))
        self.b1 = np.zeros((num_hidden))
        self.weights2 = np.random.randn(num_hidden, num_output_unit)/float(np.sqrt(num_hidden))
        self.b2 = np.zeros((num_output_unit))

        print "==> Weights Initialized"

        for line in test_lines:
            vector = [int(x) for x in line.strip().split(',')]

            self.test_vectors.append(np.array(vector))
            self.test_labels.append(np.array(vector))


        self.test_vectors = np.vstack(self.test_vectors)[:,0:-1]
        self.test_labels =  np.vstack(self.test_labels)[:,-1]


        # Make labels to categorical labels and filter out the labels
        if keep_labels is not None:
            where_keep_labels = []	
            for l in keep_labels:
                where_keep_labels.extend(list(np.where(self.train_labels==l)[0]) )

            n_data = np.sum([wkl.size for wkl in where_keep_labels])
            n_keep_labels = len(keep_labels)

            self.original_train_vectors = self.train_vectors.copy()

            self.train_vectors = np.zeros((n_data,self.num_input_nodes))

        else:
            n_data = self.train_vectors.shape[0]
            n_keep_labels = np.max(self.train_labels)+1

        self.train_labels_cat = np.zeros((n_data, n_keep_labels))
        self.filtered_labels = np.zeros((n_data))




        count = 0
        for i in range(self.train_labels.shape[0]):
            if keep_labels is not None :
                if self.train_labels[i] in keep_labels:
                    self.train_vectors[count,:] = self.original_train_vectors[i,:]

                    #self.train_labels_cat[count,self.train_labels[i]- min(keep_labels)] = 1
                    self.filtered_labels[count] = self.train_labels[i]
                    count +=1
            else:
                self.train_labels_cat[i,self.train_labels[i]] = 1

        self.train_filtered_labels_encoded = le.transform(self.filtered_labels)

        for i in range(self.train_filtered_labels_encoded.shape[0]):
            self.train_labels_cat[i, self.train_filtered_labels_encoded[i]] = 1


    def predict(self):
        # Forward Pass
        predictions = []
        for x in range(self.train_vectors.shape[0]):
            a_1 = sigmoid(np.matmul((self.weights1).T, self.train_vectors[x,:])[:,np.newaxis]+self.b1[:,np.newaxis])	

            a_2 = sigmoid(np.matmul(self.weights2.T, a_1)+self.b2[:, np.newaxis])
            predictions.append(np.argmax(a_2, axis = 0))


        predictions =  np.vstack(predictions)[:,0] 
        #

        accuracy = np.sum(predictions == self.train_filtered_labels_encoded)/float(np.shape(self.train_filtered_labels_encoded)[0])
        #pdb.set_trace()
        print accuracy




    def run(self, max_epochs=100, lr_rate=0.01):

        for i in range(max_epochs):
            train_Samples_num = self.train_vectors.shape[0]
            shuffled_indices = random.sample(range(train_Samples_num), train_Samples_num)

            for x in shuffled_indices:

                # Forward Pass
                # a_1 = sigmoid(self.train_vectors[x].dot(self.weights1)+self.b1)		

                # print self.weights1.shape, self.train_vectors[x].shape, np.matmul((self.weights1).T, self.train_vectors[x]).shape, self.b1.shape
                a_1 = sigmoid(np.matmul((self.weights1).T, self.train_vectors[x])[:,np.newaxis]+self.b1[:,np.newaxis])	
                #print a_1.shape, self.weights2.shape, self.b2[np.newaxis,:].shape

                # print a_1.shape, self.weights2.shape

                a_2 = sigmoid(np.matmul(self.weights2.T, a_1)+self.b2[:, np.newaxis])
                #print a_1.shape, a_2.shape




                # Backpropagation
                #print (self.train_labels_cat[x][:, np.newaxis] - a_2).shape, (a_2*(1-a_2)).shape
                # print (self.train_labels_cat[x][:,np.newaxis].shape), (a_2).shape,(a_2*(1-a_2)).shape			# 	#delta_2 = np.linalg.norm(self.train_labels_cat[x] - a_2)*((a_2*((1-a_2))))
                # pdb.set_trace()
                delta_2 = (a_2 - self.train_labels_cat[x][:, np.newaxis] )*(a_2*(1-a_2))
                #print delta_2.shape


                dE_dw_2 = np.matmul(a_1, delta_2.T)

                dE_db_2 = (delta_2)
                # print delta_2



            # # 	delta_1 = (a_1*(1-a_1))*(np.matmul(delta_2,self.weights2.T))
            # 	#print delta_2[:,np.newaxis].shape, self.weights2.shape
                delta_1 = (a_1*(1-a_1))*np.matmul(self.weights2, delta_2)

                dE_dw_1 = np.matmul(self.train_vectors[x][:,np.newaxis], delta_1.T)				
            #  	#print delta_1.shape, self.train_vectors[x][:,np.newaxis].shape, dE_dw_1.shape
                dE_db_1 = (delta_1)

                print delta_1

                # print dE_db_1.shape, self.b1.shape

            # 	# Update rules

            # 	#print self.weights1.shape, dE_dw_1.shape

            # 	print self.weights1.shape, self.weights2.shape




                self.weights1 = self.weights1 - lr_rate*dE_dw_1
                self.b1 = self.b1 - lr_rate*dE_db_1[:,0]	


                self.weights2 = self.weights2 - lr_rate*dE_dw_2
                self.b2 = self.b2 - lr_rate*dE_db_2[:,0]

            # print self.b1.shape, self.b2.shape
            #print self.b1

            accuracy = self.predict()




In [ ]:
if __name__ == "__main__":

	mlp = MLP(train_file, test_file)

	mlp.run()

In [14]:
run()

NameError: global name 'X_train' is not defined